# Import Libraries

# Import .exr file

In [1]:
import os
import cv2
import numpy as np
import torch
import bm3d
from bm3d import bm3d, BM3DStages
import bm3d.profiles as profiles
import OpenEXR
import Imath
import math
import itertools
import pandas as pd
import time
import warnings
from tqdm import tqdm

In [2]:
base_dir = os.getcwd()  # Current working directory

In [4]:
def process_exr(file_path):
    """
    Processes an EXR file by loading its data, adjusting exposure and gamma,
    and optionally displaying the original and adjusted images.

    Parameters:
        file_path (str): Path to the EXR file.

    Returns:
        tuple: (noisy_img, noisy_img_adjusted), where:
               - noisy_img is the original HDR image as a NumPy array.
               - noisy_img_adjusted is the adjusted image.
    """
    # Open the EXR file
    exr_file = OpenEXR.InputFile(file_path)

    # Get the header and dimensions
    header = exr_file.header()
    dw = header['dataWindow']
    width = dw.max.x - dw.min.x + 1
    height = dw.max.y - dw.min.y + 1

    # Channels to extract
    channels = ['R', 'G', 'B']

    # Determine pixel type
    pixel_type = Imath.PixelType(Imath.PixelType.HALF)

    # Allocate memory and load channels
    img = np.empty((height, width, len(channels)), dtype=np.float16)
    for i, channel in enumerate(channels):
        raw_data = exr_file.channel(channel, pixel_type)
        img[:, :, i] = np.frombuffer(raw_data, dtype=np.float16).reshape(height, width)

    # Convert to float32 for processing
    noisy_img = img.astype(np.float32)

    return noisy_img


In [6]:
def calculate_snr(image_3channel):
    """Calculate SNR (Signal-to-Noise Ratio) in decibels using mean and max-min methods."""
    # Initialize lists for SNR per channel
    snr_mean_per_channel = []
    snr_max_min_per_channel = []

    for channel in range(image_3channel.shape[2]):
        channel_data = image_3channel[:, :, channel]
        
        # Signal using max-min
        signal_max_min = channel_data.max() - channel_data.min()
        
        # Signal using mean
        signal_mean = np.mean(channel_data)
        
        # Noise (Standard deviation)
        noise = np.std(channel_data)
        
        # SNR using max-min and mean methods
        snr_max_min = signal_max_min / noise
        snr_mean = signal_mean / noise
        
        # Append to respective lists
        snr_max_min_per_channel.append(snr_max_min)
        snr_mean_per_channel.append(snr_mean)

    # Compute overall SNR for max-min and mean methods
    overall_snr_max_min = np.mean(snr_max_min_per_channel)
    overall_snr_mean = np.mean(snr_mean_per_channel)

    # Convert to decibels
    snr_max_min_db = 20 * math.log(overall_snr_max_min, 10)
    snr_mean_db = 20 * math.log(overall_snr_mean, 10)

    return snr_max_min_db, snr_mean_db


In [7]:
# Convert the NumPy array to a PyTorch tensor
def numpy_to_torch(img_np):
    # Ensure the dimensions are in the format [Channels, Height, Width]
    if len(img_np.shape) == 2:  # Grayscale image
        img_tensor = torch.tensor(img_np, dtype=torch.float32).unsqueeze(0)  # Add channel dimension
    else:  # Color image
        img_tensor = torch.tensor(img_np, dtype=torch.float32).permute(2, 0, 1).to(device)  # Rearrange dimensions

    # Normalize the tensor values to [0, 1] (optional, depending on your use case)
    img_tensor = torch.clamp(img_tensor, min=0)
    img_tensor /= 255.0
    img_tensor = img_tensor.unsqueeze(0)
    img_tensor.requires_grad = True
    
    return img_tensor

# Libraries for No Reference Metrics

In [8]:
%%capture
%pip install lpips

In [9]:
from piq import BRISQUELoss, TVLoss, CLIPIQA
import lpips

In [9]:
from contextlib import redirect_stdout

def load_lpips_model():
    with open(os.devnull, "w") as f, redirect_stdout(f):
        model = lpips.LPIPS(net='vgg').to(device)
    return model

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Choose GPU or CPU

print(f"Using device: {device}")
if device.type == "cuda":
    print(f"CUDA Device Name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. Using CPU.")

Using device: cuda
CUDA Device Name: NVIDIA GeForce RTX 2070 SUPER


In [11]:
# ignore warnings
warnings.simplefilter("ignore")

def process_bm3d(
    type,
    transform_2d_ht_names=["dct", "bior1.5"],
    max_3d_size_hts=[32],
    max_3d_size_wieners=[32],
    bs_wieners=[8],
    step_wieners=[3],
    bs_hts=[8],
    nfs=[32],
    search_window_hts=[39],
    search_window_wieners=[39],
    step_hts=[3],
    tau_matchs=[3000],
    tau_match_wieners=[400],
    lambda_thr3ds=[2.7],
    gammas=[2.0],
    beta_wieners=[2.0]
):

    # Storage for results
    results_data = []
    index = 0
    y_s = [15, 16, 17, 18, 19, 20, 21, 22]

    # Combine parameters for grid search
    parameter_combinations = itertools.product(
        y_s,
        transform_2d_ht_names,
        max_3d_size_hts,
        max_3d_size_wieners,
        bs_wieners,
        step_wieners,
        bs_hts,
        nfs,
        search_window_hts,
        search_window_wieners,
        step_hts,
        tau_matchs,
        tau_match_wieners,
        lambda_thr3ds,
        gammas,
        beta_wieners,
    )

    parameter_combinations = list(parameter_combinations)
    total_combinations = len(parameter_combinations)

    sigma_psd_values = [*range(5, 31, 5)]
    stage_args = [BM3DStages.ALL_STAGES]
    total_iterations = len(parameter_combinations) * len(stage_args) * len(sigma_psd_values)


    with tqdm(range(total_iterations), desc="Progress") as pbar:

        # Iterate through all combinations
        for i, params in enumerate(parameter_combinations):
            (
                y,
                transform_2d_ht_name,
                max_3d_size_ht,
                max_3d_size_wiener,
                bs_wiener,
                step_wiener,
                bs_ht,
                nf,
                search_window_ht,
                search_window_wiener,
                step_ht,
                tau_match,
                tau_match_wiener,
                lambda_thr3d,
                gamma,
                beta_wiener,
            ) = params

            # Configure BM3D profile
            profile = profiles.BM3DProfile()
            # Transforms used
            profile.transform_2d_ht_name = transform_2d_ht_name  # 'bior1.5'
            profile.transform_2d_wiener_name = "dct"

            # -- Exact variances for correlated noise: --

            # Variance calculation parameters
            profile.nf = nf  # 32  # domain size for FFT computations
            profile.k = 4  # how many layers of var3D to calculate

            # Block matching
            profile.gamma = gamma  # 3.0  # Block matching correction factor

            # -- Classic BM3D for correlated noise --

            # Hard-thresholding (HT) parameters:
            profile.bs_ht = bs_ht  # 8  # N1 x N1 is the block size used for the hard-thresholding (HT) filtering
            profile.step_ht = (
                step_ht  # 3# sliding step to process every next reference block
            )
            profile.max_3d_size_ht = max_3d_size_ht  # 16  # maximum number of similar blocks (maximum size of the 3rd dimension of a 3D array)
            profile.search_window_ht = search_window_ht  # 39  # side length of the search neighborhood for full-search block-matching (BM), must be odd
            profile.tau_match = (
                tau_match  # 3000  # threshold for the block-distance (d-distance)
            )

            # None in these parameters results in automatic parameter selection for them
            profile.lambda_thr3d = lambda_thr3d  # None  # 2.7  # threshold parameter for the hard-thresholding in 3D transform domain
            profile.mu2 = lambda_thr3d  # None  # 1.0

            # Wiener filtering parameters:
            profile.bs_wiener = bs_wiener  # 8
            profile.step_wiener = step_wiener  # 3
            profile.max_3d_size_wiener = max_3d_size_wiener  # 32
            profile.search_window_wiener = search_window_wiener  # 39
            profile.tau_match_wiener = tau_match_wiener  # 400
            profile.beta_wiener = beta_wiener  # 2.0
            profile.dec_level = (
                0  # dec. levels of the dyadic wavelet 2D transform for blocks
            )

            output_base_folder = f"{base_dir}/Results/Leech results/{type}/"
            os.makedirs(output_base_folder, exist_ok=True)

            file_path = f"{base_dir}/Dataset/leech/2024_04_25_11_54_01_img_x_15_y_{y}_r_0_g_1_b_0_cropped.exr"

            # Load and preprocess noisy image
            noisy_img = process_exr(file_path)  # 800x800
            noisy_img = (noisy_img[:-10, :-10] * 255).astype("uint8")

            # Process each stage and sigma_psd value
            for stage_arg in stage_args:
                for sigma_psd in sigma_psd_values:
                    # start timer
                    start_time = time.time()

                    denoised_img = bm3d(
                        noisy_img, sigma_psd=sigma_psd, stage_arg=stage_arg, profile=profile
                    )

                    end_time = time.time()

                    snr_max_min_db, snr_mean_db = calculate_snr(denoised_img)
                    img_tensor = numpy_to_torch(denoised_img)

                    # BRISQUELoss
                    try:
                        loss = BRISQUELoss(reduction="sum").to(device)
                        brsique_score = loss(img_tensor)
                        brsique_score.backward()
                        brsique_score = brsique_score.item()
                    except:
                        brsique_score = None

                    # TV
                    loss = TVLoss().to(device)
                    tv = loss(img_tensor)
                    tv.backward()
                    tv = tv.item()

                    # CLIP IQA
                    try:
                        clipiqa = CLIPIQA().to(device)
                        clipiqa_score = clipiqa(img_tensor).item()#
                    except:
                        clipiqa_score = None


                    # LPIPS
                    try:
                        noisy_img_torch = numpy_to_torch(noisy_img)
                        loss_fn = load_lpips_model()
                        lpips_score = loss_fn(noisy_img_torch, img_tensor).item()
                    except:
                        lpips_score = None

                    denoised_img_float32 = (
                        denoised_img.astype("float32") / 255.0
                    )  # Normalize to [0, 1] range
                    # OpenEXR requires the data to be split into channels (R, G, B)
                    R = denoised_img_float32[:, :, 0].tobytes()  # Red channel
                    G = denoised_img_float32[:, :, 1].tobytes()  # Green channel
                    B = denoised_img_float32[:, :, 2].tobytes()  # Blue channel

                    # Define EXR header
                    header = OpenEXR.Header(
                        denoised_img_float32.shape[1], denoised_img_float32.shape[0]
                    )  # Width, Height

                    # Write the data to an EXR file
                    path_final = output_base_folder + f"{type}-{index}_image_res.exr"
                    exr_file = OpenEXR.OutputFile(path_final, header)
                    exr_file.writePixels({"R": R, "G": G, "B": B})
                    exr_file.close()

                    # Store all parameter values and results
                    results_data.append(
                        {
                            "Coordinate y": y,
                            "Sigma PSD": sigma_psd,
                            "Stage_arg": stage_arg,
                            "Transform 2D": transform_2d_ht_name,
                            "Max 3D Size HT": max_3d_size_ht,
                            "Max 3D Size Wiener": max_3d_size_wiener,
                            "Block Size Wiener": bs_wiener,
                            "Step Wiener": step_wiener,
                            "Block Size HT": bs_ht,
                            "Step HT": step_ht,
                            "Number of Features (NF)": nf,
                            "Search Window HT": search_window_ht,
                            "Search Window Wiener": search_window_wiener,
                            "Tau Match": tau_match,
                            "Tau Match Wiener": tau_match_wiener,
                            "Lambda Thr3D": lambda_thr3d,
                            "Gamma": gamma,
                            "Beta Wiener": beta_wiener,
                            "SNR Max-Min (dB)": snr_max_min_db,
                            "SNR Mean (dB)": snr_mean_db,
                            "BRISQUE": brsique_score,
                            "Total Variation": tv,
                            "Clip-IQA": clipiqa_score,
                            "LPIPs_score" : lpips_score,
                            "Computation Time" : end_time,
                        }
                    )

                    # Convert to a DataFrame
                    pbar.set_postfix(Index = index, y = y, SNR = snr_max_min_db, BRISQUE = brsique_score, TV_loss = tv, Clip_IQA = clipiqa_score, LPIPs_score = lpips_score, Computation_Time = end_time)
                    pbar.update(1)

                    df = pd.DataFrame(results_data)
                    # Save to Excel
                    df.to_excel(
                        f"{base_dir}/Results/Leech results/{type}/results_data_{type}.xlsx",
                        index=True,
                    )
                    index = index + 1

# Influence of N patches in block

In [25]:
max_3d_size_hts=[16,32]
max_3d_size_wieners=[16,32]
process_bm3d(type="Block_Size", max_3d_size_hts=max_3d_size_hts, max_3d_size_wieners=max_3d_size_wieners)

Progress: 100%|██████████| 384/384 [1:47:50<00:00, 16.85s/it, BRISQUE=None, Clip_IQA=0.521, Computation_Time=1.74e+9, Index=383, LPIPs_score=0.00624, SNR=40.9, TV_loss=0.000519, y=22]


# Influence of Patch size hard and Patch size wien

In [26]:
bs_wieners = [8,16]
bs_hts = [8,16]
process_bm3d(type="Patch_Size", bs_hts=bs_hts, bs_wieners=bs_wieners)

Progress: 100%|██████████| 384/384 [5:44:41<00:00, 53.86s/it, BRISQUE=None, Clip_IQA=0.521, Computation_Time=1.74e+9, Index=383, LPIPs_score=0.00624, SNR=35.6, TV_loss=0.000191, y=22]


# Influence of Skip steps 1 (Reference Patches)

In [13]:
step_hts = [1]
step_wieners = [1]
process_bm3d(type="Step_size", step_hts=step_hts, step_wieners=step_wieners)

Progress: 100%|██████████| 96/96 [2:33:18<00:00, 95.81s/it, BRISQUE=None, Clip_IQA=0.521, Computation_Time=1.74e+9, Index=95, LPIPs_score=0.00624, SNR=40.7, TV_loss=0.000499, y=22] 


# Influence of Thresholds (Hard and Wiener)

In [15]:
tau_matches= [1500,3000,4500]
tau_match_wieners= [200,400,600]
process_bm3d(type="Threshold_Hard_Wiener", tau_matchs=tau_matches, tau_match_wieners=tau_match_wieners)

Progress: 100%|██████████| 864/864 [4:27:11<00:00, 18.55s/it, BRISQUE=None, Clip_IQA=0.521, Computation_Time=1.74e+9, Index=863, LPIPs_score=0.00624, SNR=40.8, TV_loss=0.000495, y=22]


# Influence of others

In [12]:
lambda_thr3ds=[2.0,2.7,3.3]
gammas=[2.0,4.0]
beta_wieners=[1.0,2.0,3.0]
process_bm3d(type="others", lambda_thr3ds=lambda_thr3ds, gammas=gammas, beta_wieners = beta_wieners)

Progress: 100%|██████████| 1728/1728 [8:40:42<00:00, 18.08s/it, BRISQUE=None, Clip_IQA=0.521, Computation_Time=1.74e+9, Index=1727, LPIPs_score=0.00624, SNR=40, TV_loss=0.000423, y=22]   


# Influence of Skip steps 5 (Reference Patches)

In [12]:
step_hts = [5]
step_wieners = [5]
process_bm3d(type="Step_size_5", step_hts=step_hts, step_wieners=step_wieners)

Progress: 100%|██████████| 96/96 [17:52<00:00, 11.17s/it, BRISQUE=None, Clip_IQA=0.521, Computation_Time=1.74e+9, Index=95, LPIPs_score=0.00624, SNR=39.8, TV_loss=0.00053, y=22] 


In [12]:
# ignore warnings
warnings.simplefilter("ignore")

def process_bm3d(
    type,
    transform_2d_ht_names=["dct", "bior1.5"],
    max_3d_size_hts=[32],
    max_3d_size_wieners=[32],
    bs_wieners=[8],
    step_wieners=[3],
    bs_hts=[8],
    nfs=[32],
    search_window_hts=[39],
    search_window_wieners=[39],
    step_hts=[3],
    tau_matchs=[3000],
    tau_match_wieners=[400],
    lambda_thr3ds=[2.7],
    gammas=[2.0],
    beta_wieners=[2.0]
):

    # Storage for results
    results_data = []
    index = 0
    y_s = [15, 16, 17, 18, 19, 20, 21, 22]

    # Combine parameters for grid search
    parameter_combinations = itertools.product(
        y_s,
        transform_2d_ht_names,
        max_3d_size_hts,
        max_3d_size_wieners,
        bs_wieners,
        step_wieners,
        bs_hts,
        nfs,
        search_window_hts,
        search_window_wieners,
        step_hts,
        tau_matchs,
        tau_match_wieners,
        lambda_thr3ds,
        gammas,
        beta_wieners,
    )

    parameter_combinations = list(parameter_combinations)
    total_combinations = len(parameter_combinations)

    sigma_psd_values = [*range(5, 31, 5)]
    stage_args = [BM3DStages.ALL_STAGES]
    total_iterations = len(parameter_combinations) * len(stage_args) * len(sigma_psd_values)


    with tqdm(range(total_iterations), desc="Progress") as pbar:

        # Iterate through all combinations
        for i, params in enumerate(parameter_combinations):
            (
                y,
                transform_2d_ht_name,
                max_3d_size_ht,
                max_3d_size_wiener,
                bs_wiener,
                step_wiener,
                bs_ht,
                nf,
                search_window_ht,
                search_window_wiener,
                step_ht,
                tau_match,
                tau_match_wiener,
                lambda_thr3d,
                gamma,
                beta_wiener,
            ) = params

            # Configure BM3D profile
            profile = profiles.BM3DProfile()
            # Transforms used
            profile.transform_2d_ht_name = transform_2d_ht_name  # 'bior1.5'
            profile.transform_2d_wiener_name = "dct"

            # -- Exact variances for correlated noise: --

            # Variance calculation parameters
            profile.nf = nf  # 32  # domain size for FFT computations
            profile.k = 4  # how many layers of var3D to calculate

            # Block matching
            profile.gamma = gamma  # 3.0  # Block matching correction factor

            # -- Classic BM3D for correlated noise --

            # Hard-thresholding (HT) parameters:
            profile.bs_ht = bs_ht  # 8  # N1 x N1 is the block size used for the hard-thresholding (HT) filtering
            profile.step_ht = (
                step_ht  # 3# sliding step to process every next reference block
            )
            profile.max_3d_size_ht = max_3d_size_ht  # 16  # maximum number of similar blocks (maximum size of the 3rd dimension of a 3D array)
            profile.search_window_ht = search_window_ht  # 39  # side length of the search neighborhood for full-search block-matching (BM), must be odd
            profile.tau_match = (
                tau_match  # 3000  # threshold for the block-distance (d-distance)
            )

            # None in these parameters results in automatic parameter selection for them
            profile.lambda_thr3d = lambda_thr3d  # None  # 2.7  # threshold parameter for the hard-thresholding in 3D transform domain
            profile.mu2 = lambda_thr3d  # None  # 1.0

            # Wiener filtering parameters:
            profile.bs_wiener = bs_wiener  # 8
            profile.step_wiener = step_wiener  # 3
            profile.max_3d_size_wiener = max_3d_size_wiener  # 32
            profile.search_window_wiener = search_window_wiener  # 39
            profile.tau_match_wiener = tau_match_wiener  # 400
            profile.beta_wiener = beta_wiener  # 2.0
            profile.dec_level = (
                0  # dec. levels of the dyadic wavelet 2D transform for blocks
            )

            output_base_folder = f"{base_dir}/Results/resTarget results/{type}/"
            os.makedirs(output_base_folder, exist_ok=True)

            file_path = f"{base_dir}/Dataset/resTarget/2024_02_09_10_17_52_img_x_15_y_{y}_r_0_g_1_b_0_cropped.exr"

            # Load and preprocess noisy image
            noisy_img = process_exr(file_path)  # 800x800
            noisy_img = (noisy_img * 255).astype("uint8")

            # Process each stage and sigma_psd value
            for stage_arg in stage_args:
                for sigma_psd in sigma_psd_values:
                    # start timer
                    start_time = time.time()

                    denoised_img = bm3d(
                        noisy_img, sigma_psd=sigma_psd, stage_arg=stage_arg, profile=profile
                    )

                    end_time = time.time()

                    snr_max_min_db, snr_mean_db = calculate_snr(denoised_img)
                    img_tensor = numpy_to_torch(denoised_img)

                    # BRISQUELoss
                    try:
                        loss = BRISQUELoss(reduction="sum").to(device)
                        brsique_score = loss(img_tensor)
                        brsique_score.backward()
                        brsique_score = brsique_score.item()
                    except:
                        brsique_score = None

                    # TV
                    loss = TVLoss().to(device)
                    tv = loss(img_tensor)
                    tv.backward()
                    tv = tv.item()

                    # CLIP IQA
                    try:
                        clipiqa = CLIPIQA().to(device)
                        clipiqa_score = clipiqa(img_tensor).item()#
                    except:
                        clipiqa_score = None


                    # LPIPS
                    try:
                        noisy_img_torch = numpy_to_torch(noisy_img)
                        loss_fn = load_lpips_model()
                        lpips_score = loss_fn(noisy_img_torch, img_tensor).item()
                    except:
                        lpips_score = None

                    denoised_img_float32 = (
                        denoised_img.astype("float32") / 255.0
                    )  # Normalize to [0, 1] range
                    # OpenEXR requires the data to be split into channels (R, G, B)
                    R = denoised_img_float32[:, :, 0].tobytes()  # Red channel
                    G = denoised_img_float32[:, :, 1].tobytes()  # Green channel
                    B = denoised_img_float32[:, :, 2].tobytes()  # Blue channel

                    # Define EXR header
                    header = OpenEXR.Header(
                        denoised_img_float32.shape[1], denoised_img_float32.shape[0]
                    )  # Width, Height

                    # Write the data to an EXR file
                    path_final = output_base_folder + f"{type}-{index}_image_res.exr"
                    exr_file = OpenEXR.OutputFile(path_final, header)
                    exr_file.writePixels({"R": R, "G": G, "B": B})
                    exr_file.close()

                    # Store all parameter values and results
                    results_data.append(
                        {
                            "Coordinate y": y,
                            "Sigma PSD": sigma_psd,
                            "Stage_arg": stage_arg,
                            "Transform 2D": transform_2d_ht_name,
                            "Max 3D Size HT": max_3d_size_ht,
                            "Max 3D Size Wiener": max_3d_size_wiener,
                            "Block Size Wiener": bs_wiener,
                            "Step Wiener": step_wiener,
                            "Block Size HT": bs_ht,
                            "Step HT": step_ht,
                            "Number of Features (NF)": nf,
                            "Search Window HT": search_window_ht,
                            "Search Window Wiener": search_window_wiener,
                            "Tau Match": tau_match,
                            "Tau Match Wiener": tau_match_wiener,
                            "Lambda Thr3D": lambda_thr3d,
                            "Gamma": gamma,
                            "Beta Wiener": beta_wiener,
                            "SNR Max-Min (dB)": snr_max_min_db,
                            "SNR Mean (dB)": snr_mean_db,
                            "BRISQUE": brsique_score,
                            "Total Variation": tv,
                            "Clip-IQA": clipiqa_score,
                            "LPIPs_score" : lpips_score,
                            "Computation Time" : end_time,
                        }
                    )

                    # Convert to a DataFrame
                    pbar.set_postfix(Index = index, y = y, SNR = snr_max_min_db, BRISQUE = brsique_score, TV_loss = tv, Clip_IQA = clipiqa_score, LPIPs_score = lpips_score, Computation_Time = end_time)
                    pbar.update(1)

                    df = pd.DataFrame(results_data)
                    # Save to Excel
                    df.to_excel(
                        f"{base_dir}/Results/resTarget results/{type}/results_data_{type}.xlsx",
                        index=True,
                    )
                    index = index + 1

In [14]:
max_3d_size_hts=[16,32]
max_3d_size_wieners=[16,32]
process_bm3d(type="Block_Size", max_3d_size_hts=max_3d_size_hts, max_3d_size_wieners=max_3d_size_wieners)

Progress: 100%|██████████| 384/384 [25:58<00:00,  4.06s/it, BRISQUE=96.1, Clip_IQA=0.325, Computation_Time=1.74e+9, Index=383, LPIPs_score=0.0493, SNR=12.7, TV_loss=0.0352, y=22]


In [13]:
bs_wieners = [8,16]
bs_hts = [8,16]
process_bm3d(type="Patch_Size", bs_hts=bs_hts, bs_wieners=bs_wieners)

Progress: 100%|██████████| 384/384 [50:28<00:00,  7.89s/it, BRISQUE=110, Clip_IQA=0.337, Computation_Time=1.74e+9, Index=383, LPIPs_score=0.0494, SNR=12.3, TV_loss=0.0265, y=22] 


In [15]:
step_hts = [1]
step_wieners = [1]
process_bm3d(type="Step_size", step_hts=step_hts, step_wieners=step_wieners)

Progress: 100%|██████████| 96/96 [11:52<00:00,  7.42s/it, BRISQUE=96.4, Clip_IQA=0.325, Computation_Time=1.74e+9, Index=95, LPIPs_score=0.0492, SNR=12.7, TV_loss=0.0344, y=22]


In [14]:
step_hts = [5]
step_wieners = [5]
process_bm3d(type="Step_size_5", step_hts=step_hts, step_wieners=step_wieners)

Progress: 100%|██████████| 96/96 [05:58<00:00,  3.74s/it, BRISQUE=101, Clip_IQA=0.325, Computation_Time=1.74e+9, Index=95, LPIPs_score=0.0492, SNR=12.8, TV_loss=0.0377, y=22] 


In [16]:
tau_matches= [1500,3000,4500]
tau_match_wieners= [200,400,600]
process_bm3d(type="Threshold_Hard_Wiener", tau_matchs=tau_matches, tau_match_wieners=tau_match_wieners)

Progress: 100%|██████████| 864/864 [1:02:35<00:00,  4.35s/it, BRISQUE=96.3, Clip_IQA=0.325, Computation_Time=1.74e+9, Index=863, LPIPs_score=0.0493, SNR=12.7, TV_loss=0.0352, y=22]


In [13]:
lambda_thr3ds=[2.0,2.7,3.3]
gammas=[2.0,4.0]
beta_wieners=[1.0,2.0,3.0]
process_bm3d(type="others", lambda_thr3ds=lambda_thr3ds, gammas=gammas, beta_wieners = beta_wieners)

Progress: 100%|██████████| 1728/1728 [1:41:42<00:00,  3.53s/it, BRISQUE=112, Clip_IQA=0.325, Computation_Time=1.74e+9, Index=1727, LPIPs_score=None, SNR=12.8, TV_loss=0.0324, y=22] 
